# Práctica 3.B - Redes Neuronales Convucionales
Hecho por:
- Jaime Benedí.
- Miguel Sevilla.

En esta parte vamos a trabajar con el dataset MNIST es un conjunto de datos en el que hay imágenes de números escritos a mano (del 0 al 9). Las dimensiones de las imágenes son de 28x28 píxeles y las imágenes están en escala de grises (1 canal). Hay diez posibles clases (los dígitos del 0 al 9). 

- Entrena y evalúa una red CNN con dos configuraciones distintas para el dataset MNIST, siguiendo el
ejemplo visto en el tutorial. Las dos configuraciones tienen que tener:
    - Un batch_size distinto.
    - Distinto número de capas convolucionales.
    - Distinto número de capas conectadas.
    - Distinto tamaño de kernel.
    - Distinto número de filtros (kernels).
    - Distintos tamaños para reducir las matrices en la capa de pooling.
    - Distintos números de neuronas en las capas ocultas de las capas conectadas.
    - Una tasa de aprendizaje distinta.
    - Distinto número de epochs.

- Consejo: primero indica explícitamente (en markdown) estos hiperpárametros elegidos para las dos
configuraciones. Después, haz el código. Explica el código en las partes donde hagas estas
configuraciones.

- Calcula el accuracy total y por clases para cada una de las configuraciones.
- Puedes dibujar una gráfica de barras para ayudarte a comparar los resultados.
- Explica las diferencias en las accuracies calculadas para ambas configuraciones y discute las razones de dichos resultados y posibles mejoras si es necesario

In [ ]:
pip install torch torchvision torchaudio

## Librerías a usar

In [ ]:
from torchvision import datasets
import torch
#import torchvision
import torchvision.transforms as transforms
import torch.utils as tutils
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#import matplotlib.pyplot as plt
#import numpy as np

## Configuración 1

##### Hiperparámetros:
- batch_size = 40.
- Número de capas convolucionales = 2
- Número de capas conectadas = 2
- Tamaño de kernel = 5
- Número de filtros (kernels) = 3
- Tamaño para reducir las matrices en la capa de pooling = 2
- Número de neuronas en las capas ocultas de las capas conectadas (2592,100), (100, 10)
- Tasa de aprendizaje = 0.1
- Número de epochs = 20

In [ ]:
lotsOfNeuronsNet_batch_size = 40
lotsOfNeuronsNet_epochs = 20
lotsOfNeuronsNet_learning_rate = 0.1

class LotsOfNeuronsNet(nn.Module):
    def __init__(self):
        super(LotsOfNeuronsNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 22, 5)
        self.conv2 = nn.Conv2d(22, 32, 5)
        self.pool1 = nn.MaxPool2d(3, 2)
        self.fc1 = nn.Linear(2592, 100)
        self.fc2 = nn.Linear(100, 10)
    
    def forward(self, x) :
        x = F.relu(self.conv1(x))
        x = self.pool1(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Configuración 2

##### Hiperparámetros:
- batch_size = 64.
- Número de capas convolucionales = 2
- Número de capas conectadas = 2
- Tamaño de kernel. 5
- Número de filtros (kernels) = 2
- Tamaño para reducir las matrices en la capa de pooling = 2
- Número de neuronas en las capas ocultas de las capas conectadas (512, 89), (89, 10)
- Tasa de aprendizaje = 0.01
- Número de epochs = 14

In [ ]:
smallNet_batch_size = 64
smallNet_epochs = 14
smallNet_learning_rate = 0.01

class SmallNet(nn.Module):
    def __init__(self):
        super(SmallNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(512, 89)
        self.fc2 = nn.Linear(89, 10)
    
    def forward(self, x) :
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Configuración de Torch y CUDA

`device` será el medio hardware que se usará para la gestión de los cálculos de la red convolucional. El coste de cómputo es alto, y esta herramienta nos da una facilidad para ello usando la GPU del sistema, siempre y cuando disponga de una tecnología CUDA.

In [50]:
TORCH_SEED=0

torch.manual_seed(TORCH_SEED)

# Modificar estos booleanos para usar CUDA o MPS en función de preferencias personales
doYouWantToUseCUDA = True
doYouWantToUseMPS = True

device = torch.device("cpu")

if doYouWantToUseCUDA and torch.cuda.is_available():
    device = torch.device("cuda")
elif doYouWantToUseMPS and torch.backends.mps.is_available():
    device = torch.device("mps")
#else:
    #device = torch.device("cpu")

## Carga de datos

In [ ]:
# Estos objetos son necesarios para transformar los datos a arrays de PyTorch
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Cargamos el dataset MNIST. Tendremos la parte de entrenamiento y la de test por separado.
# El dataset de entrenamiento tiene 60.000 imágenes y el de test 10.000 imágenes.
trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Etiquetas de las clases de MNIST, que son los números del 0 al 9
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

El siguiente objeto tendrá la función de cálculo del coste de la solución dada por el modelo. En nuestro caos, elegimos la función *entropy loss*

In [ ]:
criterion = nn.CrossEntropyLoss()

Construimos los modelos. Para ambas redes necesitamos:
- La propia red, a la cual volcaremos sobre gpu si es posible.
- Dos objetos para carga y manejo rápido de los datos de entrenamiento y prueba respectivamente, utilizados según las dinámicas de trabajo con PyTorch
- El objeto "optimizador" de la red. Éste será el encargado de realizar el ajuste de los valores de la red para que en el entrenamiento realice una optimización de la clasificación
    - Para ambos modelos utilizamos el `SGD` puesto que está pensado para imágenes, pero es necesario tener un objeto por modelo por la diferencia de parámetros de la red así como de la tasa de aprendizaje.

In [ ]:
lotsOfNeuronsNet_model = LotsOfNeuronsNet().to(device)

lotsOfNeuronsNetTrainLoader = tutils.data.DataLoader(trainset, batch_size=lotsOfNeuronsNet_batch_size, shuffle=True, num_workers=2, pin_memory=True)
lotsOfNeuronsNetTestLoader = tutils.data.DataLoader(testset, batch_size=lotsOfNeuronsNet_batch_size, shuffle=False, num_workers=2, pin_memory=True)

optimizerlotsOfNeuronsNet = optim.SGD(lotsOfNeuronsNet_model.parameters(), lr=lotsOfNeuronsNet_learning_rate, momentum=0.9)

In [ ]:
small_model = SmallNet().to(device)

smallTrainLoader = tutils.data.DataLoader(trainset, batch_size=smallNet_batch_size, shuffle=True, num_workers=2, pin_memory=True)
smallTestLoader = tutils.data.DataLoader(testset, batch_size=smallNet_batch_size, shuffle=False, num_workers=2, pin_memory=True)

optimizerSmallNet = optim.SGD(small_model.parameters(), lr=smallNet_learning_rate, momentum=0.9)

## Entrenamiento

Para el entrenamiento de cualquiera de ambas redes, seguimos la misma receta:
1. Extraemos una submuestra (batch) con los datos y la salida esperada.
2. Propagamos hacia adelante los datos por la red hasta recibir una salida.
3. Se realiza un cálculo del error cometido con esa salida respecto a la salida esperada.
4. Propagamos hacia atrás ese error por la red para ver como de alejadas están las aproximaciones en las capas, neuronas, etc. obteniendo los gradientes de los cálculos.
5. Optimizamos con esos gradientes más el algoritmo de optimización los valores de la red.
6. Repetimos ese proceso por cada batch y por cada epoch (reiniciando los valores del optimizador)

In [55]:
def train(
    model, 
    trainloader, 
    optimizer, 
    criterion, 
    epochs, 
    device=device, 
    verbose=True  
) :
    model.train()
    
    for batch_idx, (data, target) in enumerate(trainloader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if verbose and batch_idx % 100 == 0:
            print(f'Train Epoch: {epochs} [{batch_idx * len(data)}/{len(trainloader.dataset)} ({100. * batch_idx / len(trainloader):.0f}%)]\tLoss: {loss.item():.6f}')

Entrenamiento del modelo con muchas neuronas:

In [ ]:
for epoch in range(lotsOfNeuronsNet_epochs):
    train(lotsOfNeuronsNet_model, lotsOfNeuronsNetTrainLoader, optimizerlotsOfNeuronsNet, criterion, epoch, device)
    print(f"Epoch {epoch} finished\n")


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.337664
Train Epoch: 0 [4000/60000 (7%)]	Loss: 1.047639
Train Epoch: 0 [8000/60000 (13%)]	Loss: 0.716011
Train Epoch: 0 [12000/60000 (20%)]	Loss: 0.992388
Train Epoch: 0 [16000/60000 (27%)]	Loss: 0.643911
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.471602
Train Epoch: 0 [24000/60000 (40%)]	Loss: 0.736041
Train Epoch: 0 [28000/60000 (47%)]	Loss: 0.629040
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.399078
Train Epoch: 0 [36000/60000 (60%)]	Loss: 0.369858
Train Epoch: 0 [40000/60000 (67%)]	Loss: 0.393410
Train Epoch: 0 [44000/60000 (73%)]	Loss: 0.178129
Train Epoch: 0 [48000/60000 (80%)]	Loss: 0.605569
Train Epoch: 0 [52000/60000 (87%)]	Loss: 0.677212
Train Epoch: 0 [56000/60000 (93%)]	Loss: 0.529048
Epoch 0 finished

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.567550
Train Epoch: 1 [4000/60000 (7%)]	Loss: 0.432897
Train Epoch: 1 [8000/60000 (13%)]	Loss: 0.152767
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.243168
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.22283

Entrenamiento de la red pequeña:

In [ ]:
for epoch in range(smallNet_epochs):
    train(small_model, smallTrainLoader, optimizerSmallNet, criterion, epoch, device)
    print(f"Epoch {epoch} finished\n")

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.309393
Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.209870
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.290261
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.237933
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.059849
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.034393
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.067096
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.205432
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.011079
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.035760
Epoch 0 finished

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.125689
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.083335
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.055901
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.050035
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.096188
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.014877
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.149201
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.005410
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.017626
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.1

## Análisis

Con esta función, ayudándonos de la función de coste, recabamos la pérdida promedio así como contar el número de aciertos del modelo sobre los datosde prueba señalados por el `testLoader`

In [58]:
def test(
    model, 
    testloader, 
    criterion, 
    device=device, 
    verbose=True  
) :
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in testloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            _, pred = torch.max(output, 1)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    if verbose:
        print(f'\nTest set: Average loss: {test_loss / len(testloader.dataset):.4f}, Accuracy: {correct}/{len(testloader.dataset)} ({100. * correct / len(testloader.dataset):.0f}%)\n')

Con esta función, generamos predicciones para los datos y observamos el acierto cometido para cada dato respecto a la pertenencia de cada clase, lo que nos da la precisión por clase.

In [ ]:
def test_per_class(
    model,
    testloader,
    classes,
    device=device,
) :
    # diccionarios para contar las predicciones correctas y el total de predicciones por clase, inicializados a 0
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}

    with torch.no_grad():
        for data, target in testloader:
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predictions = torch.max(outputs, 1)
            
            # contamos las predicciones correctas y el total de predicciones por clase
            for label, prediction in zip(target, predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1


    # verbose per class
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

## Resultados 

In [ ]:
test(lotsOfNeuronsNet_model, lotsOfNeuronsNetTestLoader, criterion, device)
test_per_class(lotsOfNeuronsNet_model, lotsOfNeuronsNetTestLoader, classes, device)


Test set: Average loss: 0.0067, Accuracy: 9193/10000 (92%)

Accuracy for class: 0     is 94.6 %
Accuracy for class: 1     is 97.6 %
Accuracy for class: 2     is 92.1 %
Accuracy for class: 3     is 89.9 %
Accuracy for class: 4     is 90.9 %
Accuracy for class: 5     is 87.1 %
Accuracy for class: 6     is 96.1 %
Accuracy for class: 7     is 88.6 %
Accuracy for class: 8     is 93.8 %
Accuracy for class: 9     is 87.6 %


In [ ]:
test(small_model, smallTestLoader, criterion, device)
test_per_class(small_model, smallTestLoader, classes, device)


Test set: Average loss: 0.0004, Accuracy: 9924/10000 (99%)

Accuracy for class: 0     is 99.8 %
Accuracy for class: 1     is 99.8 %
Accuracy for class: 2     is 99.3 %
Accuracy for class: 3     is 99.1 %
Accuracy for class: 4     is 98.7 %
Accuracy for class: 5     is 99.0 %
Accuracy for class: 6     is 99.1 %
Accuracy for class: 7     is 99.1 %
Accuracy for class: 8     is 99.6 %
Accuracy for class: 9     is 98.8 %


## Conclusiones

Observamos entre los dos modelos como la red pequeña obtiene mejores resultados, tanto generales como por clase. Esto se puede deber a que para ambas capas convolucionales tienen su capa de pooling, lo que garantiza una mejor eliminación de ruido, así como un redimensionado más adecuado a las matrices usadas. El uso de más capas es compensado con un menor uso significativo de neuronas, lo cual afecta al coste en tiempo para bien. Su tasa de aprendizaje es baja por neurona, lo que implica que la opción correcta de evaluación del aprendizaje por cómputo es pequeña. El batch size es mayor, lo cual implica que compensa el proceso de aprendizaje por muchas secciones del dataset.